# JPL Horizons Queries

This notebook demonstrates how to use the [NASA JPL Horizons system](https://ssd.jpl.nasa.gov/horizons) to retrieve accurate planetary ephemerides data for various applications. It includes:

- How to manually construct and interpret web-based Horizons queries
- How to programmatically access Horizons data via the `requests` library
- How to programmatically access Horizons data via the `astroquery` library

This notebook complements the core example notebooks ([Example 4-1](Example_4-1.ipynb), [Example 4-2](Example_4-2.ipynb)) by providing detailed guidance on obtaining the required planetary state vectors, angular data, and rates of change using the [Horizons API](https://ssd-api.jpl.nasa.gov/doc/horizons.html).

## Example Queries using the Web
---

This section shows how to manually construct Horizons API URLs to retrieve planetary positions and rates. We highlight the key parameters to adjust (e.g., object ID, observer center, output units, and time range) and provide example queries for obtaining state vectors and apparent coordinates.

Note that you can alternatively build the queries using their web [interface](https://ssd.jpl.nasa.gov/horizons/app.html#/) instead of constructing an API URL, but the following examples focus solely on manually constructing API URLs rather than using the graphical web interface.

Refer to the [Horizons API](https://ssd-api.jpl.nasa.gov/doc/horizons.html) for more detailed information on how to form queries with different options.

### State Vectors

[Example 4-1](Example_4-1.ipynb) requires finding the position vectors of Neptune to both the Sun and Earth at the given date of May 14, 1994. The following query returns a geometric state vector of Neptune with respect to the Sun in the ICRF (J2000 equatorial) frame:

```
https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='899'&OBJ_DATA='YES'&MAKE_EPHEM='YES'&EPHEM_TYPE='VECTORS'&CENTER='500@10'&START_TIME='1994-05-14 00:00:00'&STOP_TIME='1994-05-14 00:00:01'&STEP_SIZE='1 m'&OUT_UNITS='KM-S'&REF_PLANE='FRAME'&VEC_TABLE='3'
```

➡️ [Click here to run the query](https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='899'&OBJ_DATA='YES'&MAKE_EPHEM='YES'&EPHEM_TYPE='VECTORS'&CENTER='500@10'&START_TIME='1994-05-14%2000:00:00'&STOP_TIME='1994-05-14%2000:00:01'&STEP_SIZE='1%20m'&OUT_UNITS='KM-S'&REF_PLANE='FRAME'&VEC_TABLE='3')

This query requests:
- `COMMAND='899'` → Neptune
- `CENTER='500@10'` → Heliocentric (Sun as center)
- `REF_PLANE='FRAME'` → Equatorial J2000 frame (ICRF-compatible)
- `VEC_TABLE='3'` → Cartesian state vector output
- `OUT_UNITS='KM-S'` → Kilometers and seconds
- `START_TIME` = `'1994-05-14 00:00:00'` and `STOP_TIME` = `'1994-05-14 00:00:01'` (stop time must be later than start time)
- `STEP_SIZE` = `'1 m'` (1 minute)

Scroll down to the section marked `$$SOE` to find the position vector under the labels `X`, `Y`, and `Z`. Note that we want the vector in the opposite direction (Neptune to Sun), so we'd need to flip the sign on the vector components.

To retrieve the vector from Neptune to Earth instead of the Sun, change the `CENTER` parameter to `'500@399'` (Earth-centered) in the query.<br>
➡️ [Earth-centered query example](https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND='899'&OBJ_DATA='YES'&MAKE_EPHEM='YES'&EPHEM_TYPE='VECTORS'&CENTER='500@399'&START_TIME='1994-05-14%2000:00:00'&STOP_TIME='1994-05-14%2000:00:01'&STEP_SIZE='1%20m'&OUT_UNITS='KM-S'&REF_PLANE='FRAME'&VEC_TABLE='3')

### Right Ascension, Declination, and Range

[Example 4-2](Example_4-2.ipynb) requires finding the geocentric right ascension, declination, and range - along with their rates - of Neptune at the given date and time of May 14, 1994 at an epoch time of 13:11:20.59856 UTC.

The geocentric right ascension and declination can be retrieved with the following query:

```
https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND=%27899%27&OBJ_DATA=%27NO%27&MAKE_EPHEM=%27YES%27&EPHEM_TYPE=%27OBSERVER%27&CENTER=%27500@399%27&START_TIME=%271994-05-14%2013:11:20.59856%27&STOP_TIME=%271994-05-14%2013:11:20.59857%27&STEP_SIZE=%271%20m%27&QUANTITIES=%272%27
```

This query requests:
- `COMMAND='899'` → Neptune
- `CENTER='500@399'` → Earth-centered
- `EPHEM_TYPE='OBSERVER'` → returns apparent coordinates as seen from the observer (Earth center)
- `QUANTITIES='2'` → requests apparent right ascension and declination (RA/Dec)
- `START_TIME` = `'1994-05-14 13:11:20.59856'` and `STOP_TIME` = `'1994-05-14 13:11:20.59857'` (stop time must be later than start time)
- `STEP_SIZE` = `'1 m'` (1 minute)

➡️ [Right ascension and declination angles query](https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND=%27899%27&OBJ_DATA=%27NO%27&MAKE_EPHEM=%27YES%27&EPHEM_TYPE=%27OBSERVER%27&CENTER=%27500@399%27&START_TIME=%271994-05-14%2013:11:20.59856%27&STOP_TIME=%271994-05-14%2013:11:20.59857%27&STEP_SIZE=%271%20m%27&QUANTITIES=%272%27)

Scroll down to the section marked `$$SOE` to find the right ascension (`R.A.`) and declination (`DEC`) angles. Right ascension is given in hours-minutes-seconds format and declination is given in degrees-minutes-seconds of arc format - remember to convert these to radians for any calculations using `valladopy` routines.

<details>
<summary>Click to see more queries (ra/dec rates, target range, and range rate)</summary>

We can similarly get the *rates* of the geocentric right ascension and declination by simply changing the `QUANTITIES` parameter to `'3'` instead of `'2'`:

➡️ [Right ascension and declination rates query](https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND=%27899%27&OBJ_DATA=%27NO%27&MAKE_EPHEM=%27YES%27&EPHEM_TYPE=%27OBSERVER%27&CENTER=%27500@399%27&START_TIME=%271994-05-14%2013:11:20.59856%27&STOP_TIME=%271994-05-14%2013:11:20.59857%27&STEP_SIZE=%271%20m%27&QUANTITIES=%272%27)

Scroll down to the section marked `$$SOE` to find the right ascension (`dRA*cosD`) and declination (`d(DEC)/dt`) rates. These would also need to be converted (to rad/s) for any `valladopy` routines.

The textbook example approximates these values by finding the geocentric right ascension and declination angles for May 14 and May 15, which we can find by setting `QUANTITIES='2'` to get the RA/dec angles as before, but updating the time query with: `START_TIME` = `'1994-05-14'`, `STOP_TIME` = `'1994-05-15'` and `STEP_SIZE` = `'1 d'` (1 day) to cover the timespan of one day and just retrieve the values at the beginning of the day:

➡️ [Multiple right ascension and declination angles query](https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND=%27899%27&OBJ_DATA=%27NO%27&MAKE_EPHEM=%27YES%27&EPHEM_TYPE=%27OBSERVER%27&CENTER=%27500@399%27&START_TIME=%271994-05-14%27&STOP_TIME=%271994-05-15%27&STEP_SIZE=%271%20d%27&QUANTITIES=%272%27)

Finally, we can get the target range and range rate (relative to the observer) by changing the first query to use `QUANTITIES='20'` (for information on the definitions of all quantitiy options, see the Horizons [manual](https://ssd.jpl.nasa.gov/horizons/manual.html#obsquan):

➡️ [Target range and range rate query](https://ssd.jpl.nasa.gov/api/horizons.api?format=text&COMMAND=%27899%27&OBJ_DATA=%27NO%27&MAKE_EPHEM=%27YES%27&EPHEM_TYPE=%27OBSERVER%27&CENTER=%27500@399%27&START_TIME=%271994-05-14%2013:11:20.59856%27&STOP_TIME=%271994-05-14%2013:11:20.59857%27&STEP_SIZE=%271%20m%27&QUANTITIES=%2720%27)

Where the range and range rate are given in units of AU, and km/s, respectively. Again, the textbook approximates the range rate by retrieving the data of the range values on May 14 and May 15, which we can find by altering the query as we did for the RA/dec example (and using `QUANTITIES='20'`).
</details>

## Example Query using Python and `requests`
---

This section demonstrates how to use the Python `requests` library to send queries directly to the Horizons API for better streamlined data acquisition and processing.

We start by importing the `re` and `requests` libraries. The `re` [library](https://docs.python.org/3/library/re.html) provides regex (regular expression) matching operations for strings, which we will use for parsing later.

In [1]:
import re
import requests

Next, define the base URL and query parameters, which are the same as the ones in the [State Vectors](#State-Vectors) section:

In [2]:
# Base URL
base_url = "https://ssd.jpl.nasa.gov/api/horizons.api"

# Query parameters
params = {
    "format": "json",
    "COMMAND": "'899'",  # Neptune
    "OBJ_DATA": "'YES'",
    "MAKE_EPHEM": "'YES'",
    "EPHEM_TYPE": "'VECTORS'",
    "CENTER": "'500@10'",  # Sun-centered
    "START_TIME": "'1994-05-14 00:00:00'",
    "STOP_TIME": "'1994-05-14 00:00:01'",
    "STEP_SIZE": "'1 m'",
    "OUT_UNITS": "'KM-S'",
    "REF_PLANE": "'FRAME'",
    "VEC_TABLE": "'3'"
}

Make the `GET` request:

In [3]:
response = requests.get(base_url, params=params)

Check for a successful response and extract the data from the results. Note that the data is just a plain text string that we will need to parse later.

In [4]:
if response.ok:
    data = response.json()['result']  # string
    print(data[:1000])                # preview data
else:
    print(f"Error: {response.status_code}")

*******************************************************************************
 Revised: April 22, 2021              Neptune                               899
 
 PHYSICAL DATA (update 2021-May-03):
  Mass x10^24 (kg)      = 102.409         Density (g/cm^3)       =  1.638
  Equat. radius (1 bar) = 24766+-15 km    Volume, 10^10 km^3     = 6254     
  Vol. mean radius (km) = 24624+-21       Polar radius (km)      = 24342+-30
  Geometric Albedo      = 0.41            Flattening             =  0.0171
  Sid. rot. period (III)= 16.11+-0.01 hr  Sid. rot. rate (rad/s) =  0.000108338 
  Mean solar day, h     =~16.11 h         
  GM (km^3/s^2)         = 6835099.97      GM 1-sigma (km^3/s^2)  = +-10 
  Equ. grav, ge (m/s^2) = 11.15           Pol. grav, gp (m/s^2)  = 11.41+-0.03
  Visual magnitude V(1,0)= -6.87
  Vis. mag. (opposition)=  +7.84          Obliquity to orbit     = 28.32 deg
  Sidereal orbit period = 164.788501027 y Sidereal orbit period  = 60189 d
  Mean daily motion     = 0.006020076

Now we parse the returned string by grabbing the information of interest - in this case the position vector. As noted in the Web UI section, this is under the section marked `$$SOE`:

In [5]:
# Split the text by lines
lines = data.splitlines()

# Find the $$SOE and $$EOE markers
start_idx = lines.index("$$SOE") + 1
end_idx = lines.index("$$EOE")

# Extract just the lines with ephemeris data
ephem_lines = lines[start_idx:end_idx]

# Extract the position vector (Neptune to Sun)
for line in ephem_lines:
    line = line.strip()
    if line.startswith("X ="):
        matches = re.findall(r"[-+]?\d*\.\d+E[+-]\d+", line)
        rnep_sun = [-float(val) for val in matches]  # make components negative to reverse vector direction
        break

print(f'Neptune to sun vector: {rnep_sun} km')

Neptune to sun vector: [-1666604719.851126, 3868340773.891892, 1624846812.697093] km


Here, we just covered the query from [Example 4-1](Example_4-1.ipynb), but this can easily be extended to other applications.

## Example Query using `astroquery`
---

This section demonstrates how to use the third-party Python library `astroquery` to generate the same query as the previous section and retrieve the data directly without having to parse a text block.

We first need to install `astroquery` if it doesn't exist (along with `valladopy`):

In [6]:
!pip install astroquery

Now we import the `Horizons` class and create a new object with some query parameters. Note that the class does not support all query options; see the Horizons class [documentation](https://astroquery.readthedocs.io/en/latest/api/astroquery.jplhorizons.HorizonsClass.html) for more information.

In [7]:
from astroquery.jplhorizons import Horizons

# Create the Horizons object
obj = Horizons(
    id='899',           # Neptune
    location='500@10',  # Sun-centered
    epochs={
        'start': '1994-05-14 00:00:00',
        'stop': '1994-05-14 00:00:01',
        'step': '1m'
    }
)

Query the vector ephemerides with the correct frame (ICRF) - the default `refplane` is `'ecliptic'`:

In [8]:
vectors = obj.vectors(refplane='frame')
vectors  # show table

targetname,datetime_jd,datetime_str,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str13,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64
Neptune (899),2449486.5,A.D. 1994-May-14 00:00:00.0000,11.14056444822865,-25.8582609217037,-10.86143007981392,0.002902826589042366,0.001112674989392345,0.0003826853568730526,0.1742955838565904,30.17834484359159,-1.952472041797068e-05


Now extract the x, y, and z components to form the position vector and multipy by -1 to get the vector in the desired direction (Neptune to Sun). The units (which cannot be set prior to the query) are in AU and days, so we also have to convert the quantities to kilometers:

In [9]:
import valladopy.constants as const

# Arrange position vector (multiply by -1 and convert to km)
rnep_sun = [-float(vectors[i]) * const.AU2KM  for i in ['x', 'y', 'z']]

print(f'Neptune to sun vector: {rnep_sun} km')

Neptune to sun vector: [-1666604719.8511262, 3868340773.8918924, 1624846812.6970932] km
